In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import seaborn as sns

nltk.download('stopwords')
nltk.download('punkt')

def process_msg(text):
    text = re.sub(r'(@|https?)\S+|#', '', text)
    filters = '@!"\'#$%&()*+,-/:;<=>[\\]^_`{|}~\t\n'  # '?' ve '.' kaldırıldı
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = text.lower()
    text = text.strip()

    
    stop_words = set(stopwords.words('turkish'))
    words = word_tokenize(text)
    words_filtered = [word for word in words if word.isalpha() and word not in stop_words]
    text = ' '.join(words_filtered)
    
    return text

def classify_message(message, vectorizer, models):
   
    processed_message = process_msg(message)
    
   
    X_message = vectorizer.transform([processed_message])
    
    
    predictions = {}
    for model_name, model in models.items():
        prediction = model.predict(X_message)
        predictions[model_name] = prediction[0]
        
    return predictions


xt = pd.read_excel(r'C:\Users\LENOVO\Desktop\TeknoFest\half29072024_teknofest.xlsx')
xt = xt.dropna()
xt['mesaj'] = xt['mesaj'].astype(str).apply(process_msg)
xt['label'] = xt['label']


print(xt.head())


plt.figure(figsize=(10, 6))
xt['label'].value_counts().plot(kind='bar')
plt.title('Label Dağılımı')
plt.xlabel('Label')
plt.ylabel('Frekans')
plt.show()


print(xt['label'].value_counts())


vectorizer = TfidfVectorizer()


X = vectorizer.fit_transform(xt['mesaj'])
y = xt['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


models = {
    "SVM": SVC(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Logistic Regression": LogisticRegression()
}


for model_name, model in models.items():
    print(f"Model: {model_name}")
    model.fit(X_train, y_train)
    y_pred = cross_val_predict(model, X_test, y_test, cv=5)
    
    
    print(classification_report(y_test, y_pred))
    
    
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    print("\n")
message = input("Sınıflandırmak istediğiniz mesajı girin: ")
predictions = classify_message(message, vectorizer, models)
print("Tahminler:")
for model_name, prediction in predictions.items():
    print(f"{model_name}: {prediction}")
